In [4]:
import requests
import gmaps
import pandas as pd
from gmaps import Geocoding

In [5]:
def cab_details_avai_ola_src(ride):
    columns = ['Car type', 'Availability', 'Number of seats', 'ETA', 'Distance' ]
    ride_cat = ride['categories']
    dicti = []
    for i in range(0,len(ride_cat)):
        seats = "4"
        avail = "Yes"
        if ride_cat[i]['eta'] == -1:
            avail = "No"
            data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail}
            dicti.append(data)
        else:
            if ride_cat[i]['display_name']=="Share":
                seats = "1/2"
                data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail, 'Number of seats':           seats,'ETA' : ride_cat[i]['eta'],'Distance' : ride_cat[i]['distance']}
                dicti.append(data)   
            else:
                data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail, 'Number of seats': seats,'ETA' : ride_cat[i]['eta'],'Distance' : ride_cat[i]['distance']}
                dicti.append(data)
    df = pd.DataFrame(dicti,columns = columns)
    return df

In [6]:
def cab_details_ola_src_dst(ride):
    columns = ['Car type', 'Availability', 'Number of seats', 'Distance(Km)', 'Estimated Travel time(min)', 'Minimum Price', 'Maximum Price','ETA' ]
    #df = pd.DataFrame(data,columns = columns)
    ride_cat = ride['categories']
    ride_est = ride['ride_estimate']
    a_list = []
    for i in range(0,len(ride_cat)):
        if ride_cat[i]['eta'] == 0:
            print (ride_cat[i]['display_name'] + " not available")
            data1 = {'Car type' : ride_cat[i]['display_name'], 'Availability': "No"}
            a_list.append(data1)
        else:
            avail='Yes'
            if ride_cat[i]['eta'] == -1: avail='No'
            if ride_cat[i]['display_name'] == "Share":
                for j in range(0,2):
                    data1 = {'Car type': ride_cat[i]['display_name'],
                             'Number of seats': ride_est[i]['fares'][j]['seats'], 
                             'Travel time(min)': ride_est[i]['travel_time_min'], 
                             'Maximum Price': ride_est[i]['fares'][j]['cost'], 
                             'Minimum Price': ride_est[i]['fares'][j]['cost'], 
                             'Availability' : avail,
                             'ETA': ride_cat[i]['eta']}
                    a_list.append(data1)
            else:
                data1 = {'Car type': ride_cat[i]['display_name'],
                         'Number of seats': "4",
                         'Distance(Km)': ride_est[i]['distance'], 
                         'Estimated Travel time(min)': ride_est[i]['travel_time_in_minutes'], 
                         'Maximum Price': ride_est[i]['amount_max'], 
                         'Minimum Price': ride_est[i]['amount_min'], 
                         'Availability' : avail,
                         'ETA': ride_cat[i]['eta']}
                a_list.append(data1)          
    df = pd.DataFrame(a_list,columns = columns)
    return df

In [8]:
def look_ola(source, destination=None):
    api = Geocoding()
    ### Check validity of Source.
    try:
        source_loc = api.geocode(source)
    except:
        return ('Enter Valid Source.')
    
    source_lat_long = source_loc[0]['geometry']['location']
    ### Check if destination is provided or not.
    if destination : 
        ### Check Validity of Destination.
        try:
            destination_loc = api.geocode(destination)
        except:
            return ('Enter Valid destination.')
        
        destination_lat_long = destination_loc[0]['geometry']['location']
        ### If destination is provided then set latitude and lognitude of Source and Destination both.
        payload = {'pickup_lat': str(source_lat_long['lat']), 'pickup_lng':str(source_lat_long['lng']), 'drop_lat':str(destination_lat_long['lat']), 'drop_lng':str(destination_lat_long['lng']), 'pickup_time': '19/4/2017 16:30'}
    
    else:
        ### If destination is not provided then set latitude and lognitude of Source only.
        payload = {'pickup_lat': str(source_lat_long['lat']), 'pickup_lng':str(source_lat_long['lng']), 'pickup_time': '19/4/2017 16:30'}
    
    headers = {'X-APP-TOKEN' : "ab8a1c87e9f64ba3a36c2858ad9c08c2"} ### Server Token provided by API.
    
    response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)
    
    if destination : 
        ### If destination is available.
            ### Check if any error occured or not.
        if 'categories' in response.json().keys():
            ### If there is no error then return the Cab Details in the form of DataFrame which is created from json file. 
            cab_details_df_ola_src_dst=cab_details_ola_src_dst(response.json())
            cab_details_df_ola_src_dst.fillna(0,axis=1,inplace=True)
            return cab_details_df_ola_src_dst
        else:
            return response.json()['message']
    
    else:
         ### If destination is not available.
            ### Check if any error occured or not.
        if 'categories' in response.json().keys():
            ### If there is no error then return the Cab Availability Details in the form of DataFrame which is created from json file.
            cab_details_df_ola_src = cab_details_avai_ola_src(response.json())
            cab_details_df_ola_src.fillna(0,axis=1,inplace=True)
            return cab_details_df_ola_src
        else:
            return response.json()['message']


In [19]:
def book_ola(source,destination,cabtype,noofseats):
    df=look(source,destination)
    driver_name=["Tharoon Veerasethu","Sharmaji","Chachaji","Chunnu","Munni","beta","daadaji","daadi","bhai","shinde","langoor"]
    for i in range(df.shape[0]):
        x=list(df.loc[i])
        if cabtype in x:
            for j in x:
                if j == cabtype and df.loc[i,'Availability']=='Yes':
                    SessInfo['bookingid']="OLA"+str(random_with_N_digits(6))
                    SessInfo['drivername']=random.choice(driver_name)
                    SessInfo['driverno']=random_with_N_digits(10)
                    SessInfo['carno']="KN-"+str(random_with_N_digits(2))+"-"+str(random_with_N_digits(4))
                    SessInfo['fare']=(df.loc[i,'Minimum Price']+df.loc[i,'Maximum Price'])/2
                    SessInfo['cabtype']=cabtype
                    SessInfo['noofseats']=noofseats
                    SessInfo['eta']=df.loc[i,'ETA']
                    if cabtype != 'Share': 
                        SessInfo['distance']=df.loc[i,'Distance(Km)']
                        Sessinfo['ett']=df.loc[i,'Estimated Travel time(min)']
                    break  

In [17]:
x=look_ola("K R Puram","stackroute koramangala")

In [18]:
x

,Car type,Availability,Number of seats,Distance(Km),Estimated Travel time(min),Minimum Price,Maximum Price,ETA
0,Micro,Yes,4,15.76,62.0,246,282,1
1,Mini,Yes,4,15.76,62.0,334,384,1
2,Sedan,Yes,4,15.76,62.0,336,386,1
3,Prime Sedan,No,4,15.76,62.0,274,315,-1
4,Share,No,1,0.00,0.0,130,130,-1
5,Share,No,2,0.00,0.0,143,143,-1
